### Imports + Generator (PixelShuffle)

In [1]:
import math
import torch
import torch.nn as nn

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, n_feats: int):
        super().__init__()
        # Residual block
        self.block = nn.Sequential(
            nn.Conv2d(n_feats, n_feats, 3, padding=1),  # First conv
            nn.ReLU(inplace=True),
            nn.Conv2d(n_feats, n_feats, 3, padding=1),  # Second conv
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.block(x)